In [1]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/MyDrive/CorpusforDDL_compute')
print(os.getcwd())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/CorpusforDDL_compute


In [2]:
import pandas as pd
import re

# 國語日報新聞

In [3]:
# 讀入國語日報新聞 txt 檔
mandarin_daily_path = 'CorpusforDDL/國語日報新聞/MandarinDailyNews-20181217'
all_files = os.listdir(mandarin_daily_path)

mandarin_daily_corpus = []
for f in all_files:
  with open(f'{mandarin_daily_path}/{f}') as f:
      lines = f.readlines()
      mandarin_daily_corpus.append(lines)

In [4]:
# 前處理
def preprocess_mandarin_daily(string):

  url = r'(www|http|ftp|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?'

  clean_string = re.sub(url, '', string) # 移除 url
  clean_string = re.sub('[\n△◎]', '', clean_string) # 移除換行符號和△◎符號
  clean_string = re.sub("【\w+】", "", clean_string) # 移除【記者xxx報導】
  splitted_strings = re.split(r'(?<=[。！？])', clean_string) # 將長句子用。！？斷開
  splitted_strings = [x for x in splitted_strings if len(x)>1] # 濾掉空字串和只有一個標點符號的字串

  return splitted_strings

In [5]:
test = """
【記者楊惠芳報導】
△小說家徐薏藍每年固定出兩本書之外，還在春秋兩季，
◎邀約文友，組成旅行團，周遊列國，觀賞世界各地美景。
今年開春後，她準備率團去帛琉，一窺海底祕密。目前報名的有五對夫婦及六位文友。
欲參加文友請早報名，以免向隅。
http://www.lttc.ntu.edu.tw。
"""

print(preprocess_mandarin_daily(test))

['小說家徐薏藍每年固定出兩本書之外，還在春秋兩季，邀約文友，組成旅行團，周遊列國，觀賞世界各地美景。', '今年開春後，她準備率團去帛琉，一窺海底祕密。', '目前報名的有五對夫婦及六位文友。', '欲參加文友請早報名，以免向隅。']


In [6]:
mandarin_daily_corpus_preprocessed = []

for text in mandarin_daily_corpus:
  text_preprocessed = []
  for string in text:
    check_punctuation = re.search(r'[，。！？]', string) # 檢查字串是否包含，。！？
    check_chinese = re.search(r'[\u4e00-\u9fff]+', string) # 檢查字串是否包含中文字

    if check_punctuation and check_chinese:
      preprocessed = preprocess_mandarin_daily(string)
      text_preprocessed.append(preprocessed)
  mandarin_daily_corpus_preprocessed.append(text_preprocessed)

In [8]:
mandarin_daily_corpus_preprocessed[0][:5]

[['今天是中華民國九十年元旦，也是二十一世紀的第一天，陳水扁總統發表跨世紀談話，並且向全民祝賀新年。',
  '陳總統表示，過去政府的「戒急用忍」政策有當時的背景及其必要，但是未來政府將以「積極開放，有效管理」的新視野，規畫我們的新世紀經貿版圖。'],
 ['陳總統指出，二十一世紀，經濟競爭力依然是我們賴以生存的命脈。',
  '面對全球景氣循環的低潮以及包括美國、日本等國家經濟成長的向下修正，臺灣地區的經濟繁榮將再次面臨高難度的考驗，政府與民間要為最壞的可能做最好的準備。'],
 ['另外，因應我們即將加入世界貿易組織，陳總統說，政府有責任面對全新的經濟情勢，將包括兩岸經貿在內的各項課題，重新納入全球市場的考量。',
  '他說，在過去的「戒急用忍」之外，未來我們將以「積極開放，有效管理」的新視野，在知識經濟的既定方針之下，為新世紀經貿版圖做出宏觀的規畫，並且逐步加以落實。'],
 ['陳水扁總統說，面對過去半年多以來的政經情勢，國人應該都能深刻的體會，有穩定的政局才有經濟的繁榮，有穩定的政局也才有改革和進步。'],
 ['在兩岸問題方面，陳總統強調，依據中華民國憲法，「一個中國」原本並不是問題，也希望對岸能夠深入了解臺灣地區人民心中的疑慮在哪裡，尊重兩千三百多萬人民的意志。']]

In [88]:
len(mandarin_daily_corpus) == len(mandarin_daily_corpus_preprocessed)

True

# ASBC

In [9]:
# 讀入 ASBC txt 檔
asbc_path = 'CorpusforDDL/ASBC_去XML標記'
all_files = os.listdir(asbc_path)

asbc_corpus = []
for f in all_files:
  with open(f'{asbc_path}/{f}') as f:
      lines = f.readlines()
      asbc_corpus.append(lines)

In [52]:
# 前處理
import unicodedata
def preprocess_asbc(string):

  clean_string = unicodedata.normalize('NFKC', string) # 全形轉半形
  clean_string = re.sub(r'\n', '', clean_string) # 移除換行符號

  clean_string = clean_string.translate(str.maketrans({',': '，', 
                                                       '!': '！', 
                                                       '?': '？'})) # ，！？改回全形
  return clean_string

In [60]:
asbc_corpus_preprocessed = []
for text in asbc_corpus:
  text_preprocessed = []
  for string in text:
    preprocessed = preprocess_asbc(string)
    joined = ''.join(preprocessed) # 將所有字串合成一長串
    splitted = re.split(r'(?<=[。！？])', joined) # 將長句子用。！？斷開
    text_preprocessed.append(splitted)
  asbc_corpus_preprocessed.append(text_preprocessed)

In [92]:
asbc_corpus_preprocessed = []

for text in asbc_corpus:
  preprocessed = [preprocess_asbc(string) for string in test]
  joined = ''.join(preprocessed)
  splitted = re.split(r'(?<=[。！？])', joined)
  asbc_corpus_preprocessed.append(splitted)

In [93]:
asbc_corpus_preprocessed[0][:5]

['碰到(VC) 這(Nep) 種(Nf) 情形(Na) ，(COMMACATEGORY)真(D) 是(SHI) 會(D) 令(VL) 人(Na) 又(Caa) 氣憤(VH) 又(Caa) 苦惱(VH) 。',
 '(PERIODCATEGORY)相信(VK) 大多數(Neqa) 的(DE) 母親(Na) 也(D) 都(D) 有(V_2) 過(Di) 類似(VG) 的(DE) 困擾(Na) ，(COMMACATEGORY)因為(Cbb) 我們(Nh) 的(DE) 生活(Na) 環境(Na) 逐漸(D) 被(P) 色情(Na) 電影(Na) 、(PAUSECATEGORY) 錄影帶(Na) 、(PAUSECATEGORY) 畫片(Na) 、(PAUSECATEGORY) 書刊(Na) 所(D) 污染(VC) 。',
 '(PERIODCATEGORY)尤其(D) 在(P) 「(PARENTHESISCATEGORY) 笑(VC) 貧(VH) 不(D) 笑(VA) 娼(Na) 」(PARENTHESISCATEGORY) 的(DE) 社會(Na) 風氣(Na) 裡(Ng) ，(COMMACATEGORY)有(V_2) 些(Nf) 唯利是圖(VH) 的(DE) 不法(A) 商人(Na) ，(COMMACATEGORY)更(D) 是(SHI) 以(P) 誇張(VHC) 的(DE) 色情(Na) 表演(Na) 和(Caa) 刺激(VH) 的(DE) 畫面(Na) 來(D) 招攬(VC) 生意(Na) 。',
 '(PERIODCATEGORY)要(D) 我們(Nh) 這些(Neqa) 做(VG) 父母(Na) 的(DE) 完全(D) 禁止(VE) 孩子(Na) 們(Na) 去(D) 接觸(VC) ，(COMMACATEGORY)可(D) 真(D) 是(SHI) 防不勝防(VH) 。',
 '(PERIODCATEGORY)縱使(Cbb) 你(Nh) 的(DE) 孩子(Na) 現在(Nd) 沒(D) 看(VC) 過(Di) ，(COMMACATEGORY)以後(Nd) 他(Nh) 同樣(VH) 會(D) 有(V_2) 機會(Na) 接觸(VC) ，(COMMACATEGORY)所以(Cbb) ，(COMMACATEGORY)我們(Nh) 要(D) 做(VC) 的(DE) 並(D

In [94]:
len(asbc_corpus) == len(asbc_corpus_preprocessed)

True

# EDsplit

In [99]:
# # 讀入 EDsplit txt 檔
# edsplit_path = 'CorpusforDDL/EDsplit-1File1Days-DeleteRepeatPattern-TitleWithoutPeriod-out'
# all_files = os.listdir(edsplit_path)

# edsplit_corpus = []
# for f in all_files:
#   with open(f'{edsplit_path}/{f}', encoding= 'unicode_escape') as f:
#       lines = f.readlines()
#       edsplit_corpus.append(lines)